In [1]:
import os
from triangulator import create_triangulator_from_files

import sys
sys.path.append('../')
from utils.file_utils import create_capture_from_json

calibration_folder = '../calibration/calibration_data/'
intr_L = os.path.join(calibration_folder, 'calibration_L_frames=623_error=1.5265531081871597.pkl')
intr_R = os.path.join(calibration_folder, 'calibration_R_frames=624_error=0.99089142253755.pkl')
orientation_L = '../calibration/orientation_video_L.mp4.pkl'
orientation_R = '../calibration/orientation_video_R.mp4.pkl'

triangulator = create_triangulator_from_files(
    intr_files=[intr_L, intr_R],
    orientation_files=[orientation_L, orientation_R]
)

Image size is (1920, 1080)
Image size is (1920, 1080)
AAAAAAAAAAAAAAAAAAAAAAAAAAa
[array([[ 1.30658287],
       [-0.24183669],
       [-0.36029694]]), array([[0.8370347 ],
       [0.50877275],
       [1.42079534]])]


In [3]:
zero_point_L = (834, 751)
zero_point_R = (767, 887)

triangulator.triangulate([zero_point_L, zero_point_R])
position_q = (triangulator.projection_matrices[0] @ [0, 0, 0, 1])
position_q = position_q / position_q[-1]
print(position_q)
extr_vec = triangulator.extrinsics[0] @ [0, 0, 0, 1]
print(triangulator.camera_matrices[0] @ extr_vec[0:3])

[833.46269592 753.58957361   1.        ]
[ 1.38878569e+05  8.34472582e+05 -4.13995407e+02]


In [7]:
import cv2
import numpy as np
print(np.linalg.inv(triangulator.camera_matrices[0]) @ triangulator.projection_matrices[0])
rv_matr, _ = cv2.Rodrigues(triangulator.rvecs[0])
print(-rv_matr.T @ triangulator.tvecs[0].T)

[[ 9.19813463e-01  1.22229568e-01 -3.72831231e-01  3.92856688e+02]
 [-3.91347663e-01  2.17730418e-01 -8.94114351e-01  7.78937189e+02]
 [-2.81105116e-02  9.68325048e-01  2.48105625e-01 -4.13995407e+02]]
[[-68.15724473]
 [183.26509932]
 [945.64275085]]


In [5]:
caps = [
    create_capture_from_json(4, '../config/capture_params.json'),
    create_capture_from_json(2, '../config/capture_params.json')
]

[ WARN:0@1422.668] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video4): can't open camera by index
[ WARN:0@1422.668] global cap.cpp:342 open VIDEOIO(V4L2): backend is generally available but can't be used to capture by index
[ WARN:0@1422.669] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video2): can't open camera by index
[ WARN:0@1422.669] global cap.cpp:342 open VIDEOIO(V4L2): backend is generally available but can't be used to capture by index


In [47]:
import cv2


while True:
    idx = 0
    for cap in caps:
        ret, frame = cap.read()

        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners, ids, _ = cv2.aruco.detectMarkers(
            image=gray,
            dictionary=cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
        )
        if ids is not None and 0 in ids:
            for corner_idx in range(len(ids)):
                if ids[corner_idx] == 0:
                    cv2.aruco.drawDetectedMarkers(
                        frame, 
                        corners=corners, 
                        ids=ids
                        )
                    break
        idx += 1
        cv2.imshow(f'cam{idx}', cv2.resize(src=frame, dsize=None, fx=0.35, fy=0.35))
    if cv2.waitKey(10) == 27:
        break


cv2.imshow('huh', frame)
cv2.waitKey(1000)
cv2.destroyAllWindows()

In [48]:
for cap in caps:
    cap.release()

cv2.destroyAllWindows()